In [2]:
import xarray as xr
import pandas as pd
import geopandas as gpd

def prep_nw_data(
    path, file, length_time, env_param, scenario, min_lat=None, max_lat=None, min_lon=None, max_lon=None, all_cells=False):
    """
    ### This code is only used on the NCAR cluster. ###

    Reads the nuclear war data from Cheryls workspace.
    Creates a geopandas dataframe for it for a given
    environmental parameter and saves it in cwd as a pickle.

    Arguments:
        path: path to the file
        file: file name
        min_lon: index of the minimal longitude to sample from
        max_lon: index of the maximal longitude to sample from
        min_lat: index of the minimal latitude to sample from
        max_lat: index of the maximal latitude to sample from
        length_time: how much of the original dataset should
                     be used. Measured in month, max = 300
        env_param: the environmental parameter to look at
        all_cells: if True, all cells are used, if False, only selection
    Returns:
        None
    """
    # Read in the data
    ds = xr.open_dataset(path + file)
    # 0 here means we are only using the uppermost layer of the ocean
    if all_cells:
        env_time = ds[env_param][:length_time, 0, :, :]    
    else:
        env_time = ds[env_param][:length_time, 0, min_lat:max_lat, min_lon:max_lon]
    # Make it a dataframe
    env_time_df = env_time.to_dataframe()
    # Delete the depth column, as it is not needed
    if env_param == "PAR_avg":
        del env_time_df["z_t_150m"]
    else:
        del env_time_df["z_t"]
    # Create a new index to remove redundant information
    env_time_df.reset_index(inplace=True)
    env_time_df.set_index(["time", "TLONG", "TLAT"], inplace=True)
    # delte the nlat and nlon columns, as thy are not needed anymore
    del env_time_df["nlat"]
    del env_time_df["nlon"]
    # remove all columns that are only nan
    env_time_df.dropna(axis=0, how="all", inplace=True)
    # Save to pickle
    env_time_df.to_pickle(
        scenario + "/nw_" + env_param + "_" + str(length_time) + "_months_" + scenario + ".pkl"
    )

In [3]:
# Do it for the big nulcear war
env_params = ["TEMP", "SALT", "PO4", "NO3", "PAR_surf", "NH4", "Fe"]
scenario = "150tg"
for env_param in env_params:
    print(env_param)
    path = '/glade/u/home/chsharri/Work/NW/'
    file = 'nw_ur_150_07.pop.h.' + env_param + '.nc'
    # Index positions of the US in the dataset
    min_lat = 250
    max_lat = 320
    min_lon = 235
    max_lon = 300
    length_time = 36
    if env_param == "PAR_surf":
        env_param = "PAR_avg"
    prep_nw_data(path, file, length_time, env_param, scenario, min_lat, max_lat, min_lon, max_lon)
    prep_nw_data(path, file, 120, env_param, scenario, all_cells=True)
    
print("done")

TEMP
SALT
PO4
NO3
PAR_surf
NH4
Fe
done


In [4]:
# Do it for the smaller nuclear wars
env_params = ["TEMP", "SALT", "NO3", "PAR_surf", "NH4", "Fe", "PO4"]
for scenario, target in {"5tg":"01", "16tg":"04", "27tg":"02", "37tg":"03", "47tg":"05"}.items():
    print()
    print(scenario)
    for env_param in env_params:
        print(env_param)
        path = '/glade/u/home/chsharri/Work/NW/'
        file = "nw_targets_" + target + ".pop.h." + env_param + '.nc'
        # Index positions of the US in the dataset
        min_lat = 250
        max_lat = 320
        min_lon = 235
        max_lon = 300
        length_time = 36
        if env_param == "PAR_surf":
            env_param = "PAR_avg"
        # prep_nw_data(path, file, length_time, env_param, scenario, min_lat, max_lat, min_lon, max_lon)
        prep_nw_data(path, file, 120, env_param, scenario, all_cells=True)
    
print("done")


5tg
TEMP
SALT
NO3
PAR_surf
NH4
Fe
PO4

16tg
TEMP
SALT
NO3
PAR_surf
NH4
Fe
PO4

27tg
TEMP
SALT
NO3
PAR_surf
NH4
Fe
PO4

37tg
TEMP
SALT
NO3
PAR_surf
NH4
Fe
PO4

47tg
TEMP
SALT
NO3
PAR_surf
NH4
Fe
PO4
done


In [ ]:
# Do it for the control
env_params = ["TEMP", "SALT", "PO4", "NO3", "PAR_surf", "NH4", "Fe"]
for env_param in env_params:
    print(env_param)
    path = '/glade/u/home/chsharri/Work/NW/'
    file = "nw_cntrl_03.pop.h." + env_param + '.nc'
    # Index positions of the US in the dataset
    min_lat = 250
    max_lat = 320
    min_lon = 235
    max_lon = 300
    length_time = 36
    if env_param == "PAR_surf":
        env_param = "PAR_avg"
    #prep_nw_data(path, file, length_time, env_param, "control", min_lat, max_lat, min_lon, max_lon)
    prep_nw_data(path, file, 120, env_param, "control", all_cells=True)
    
print("done")

TEMP
SALT
PO4
NO3
PAR_surf
NH4
Fe


# Getting the area

In [18]:
def get_area(path, file):
    """
    Gets the file with all the areas for grid_cells and saves it as a csv
    Arguments:
        path: path to the file
        file: filename
    Returns:
        None
    """
    data_set = xr.open_mfdataset(casePath + file)
    area = data_set["TAREA"][0,:,:]
    area = area.to_dataframe()
    # convert from cm² to km²
    area["TAREA"] = area["TAREA"] / 1e+10
    area = area.reset_index()
    area = area.set_index(["TLAT", "TLONG"])
    area = area["TAREA"].to_frame()
    area.to_csv("area_grid.csv", sep=";")

In [19]:
#Opening data
casePath = '/glade/campaign/cgd/projects/cchen/MCB/b.e21.BSSP245smbb.f09_g17.MCB-050PCT.001/ocn/hist/'
file = 'b.e21.BSSP245smbb.f09_g17.MCB-050PCT.001.pop.h.2035*.nc'
get_area(casePath, file)